In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape

(60000, 28, 28)

## Channel
### 因為一開始設定灰階，所以只有一個channel，所以要轉換資料格式

In [5]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [6]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [7]:
x_train[87].shape

(28, 28, 1)

In [8]:
y_train[87]

9

In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## 讀入必要的函式

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

## Step 1 打造函數學習機 (CNN)
### 改變filter數量

In [12]:
model = Sequential()

In [13]:
model.add(Conv2D(32, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [14]:
# 輸出32個 28x28 矩陣
# 事實上是(28, 28, 32)

In [15]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
# (14,14,32)

In [17]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [18]:
# output (14, 14, 64)

In [19]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [20]:
# output (7, 7, 64)

In [21]:
model.add(Conv2D(128, (3,3), padding='same',
                activation='relu'))

In [22]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [23]:
model.add(Flatten())

In [24]:
model.add(Dense(54, activation='relu'))

In [25]:
model.add(Dense(10, activation='softmax'))

## 檢視神經網路

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         36992     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [27]:
# 3*3 (權重) + 1 (bias)

(3*3+1)*32

320

In [28]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

## Step 2. fit
### batch_size改成256, epochs改成64

In [29]:
model.fit(x_train, y_train, batch_size=256, epochs=64)

Train on 60000 samples
Epoch 1/64
60000/60000 [==============================] - 54s 905us/sample - loss: 0.0898 - accuracy: 0.1802
Epoch 2/64
60000/60000 [==============================] - 50s 832us/sample - loss: 0.0896 - accuracy: 0.3346
Epoch 3/64
60000/60000 [==============================] - 51s 844us/sample - loss: 0.0892 - accuracy: 0.4193
Epoch 4/64
60000/60000 [==============================] - 54s 897us/sample - loss: 0.0887 - accuracy: 0.4322
Epoch 5/64
60000/60000 [==============================] - 54s 902us/sample - loss: 0.0878 - accuracy: 0.3895
Epoch 6/64
60000/60000 [==============================] - 54s 903us/sample - loss: 0.0855 - accuracy: 0.3636
Epoch 7/64
60000/60000 [==============================] - 54s 903us/sample - loss: 0.0758 - accuracy: 0.4243
Epoch 8/64
60000/60000 [==============================] - 54s 898us/sample - loss: 0.0473 - accuracy: 0.7379
Epoch 9/64
60000/60000 [==============================] - 54s 894us/sample - loss: 0.0251 - accuracy: 0.8

## Step 3. 預測

In [30]:
result = model.predict_classes(x_test)

In [31]:
def my_predict(n):
    print('CNN的預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [32]:
from ipywidgets import interact_manual

In [33]:
interact_manual(my_predict, n=(0, 9999))

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

<function __main__.my_predict(n)>

In [34]:
score = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 5s 522us/sample - loss: 0.0026 - accuracy: 0.9822


In [35]:
loss, acc = score

In [36]:
print('測試資料的正確率是', acc)

測試資料的正確率是 0.9822


## 把model存起來

In [37]:
model.save('fashionCNNmodel.h5')